In [22]:
# 0.0 import packages

import numpy as np
import time

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as func
from numba import cuda as numba
from GPUtil import showUtilization as gpu_usage

import matplotlib.pyplot as plt

In [23]:
# 0.2 GPU stuff

# print ("cuda: ", torch.cuda.is_available())
# device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

# print ("current device: ", device)
# print ("count: ", torch.cuda.device_count())

# # function to clear GPU memory
# def free_gpu_cache():
#     print("Initial GPU Usage")
#     gpu_usage()                             
#     torch.cuda.empty_cache()
#     numba.select_device(0)
#     numba.close()
#     numba.select_device(0)
#     print("GPU Usage after emptying the cache")
#     gpu_usage()

# if torch.cuda.is_available(): 
#     free_gpu_cache()

In [24]:
# 1.0 load in datasets

train_feats = np.load("data/train_feats.npy", allow_pickle=True)
train_labels = np.load("data/train_labels.npy", allow_pickle=True)
test_feats = np.load("data/test_feats.npy", allow_pickle=True)
test_labels = np.load("data/test_labels.npy", allow_pickle=True)
val_feats = np.load("data/val_feats.npy", allow_pickle=True)
val_labels = np.load("data/val_labels.npy", allow_pickle=True)

# print ("train feats: ", train_feats.shape, " type: ", type(train_feats))
# print ("train labels: ", train_labels.shape, " type: ", type(train_labels))

# print ("test feats: ", test_feats.shape, " type: ", type(test_feats))
# print ("test labels: ", test_labels.shape, " type: ", type(test_labels))

# print ("val feats: ", val_feats.shape, " type: ", type(val_feats))
# print ("val labels: ", val_labels.shape, " type: ", type(val_labels))

# create tensors
train_feats_tensor = torch.tensor(train_feats, requires_grad=True, dtype=torch.float)
train_labels_tensor = torch.tensor(train_labels, dtype=torch.int)

test_feats_tensor = torch.tensor(test_feats, requires_grad=True, dtype=torch.float)
test_labels_tensor = torch.tensor(test_labels, dtype=torch.int)

val_feats_tensor = torch.tensor(val_feats, requires_grad=True, dtype=torch.float)
val_labels_tensor = torch.tensor(val_labels, dtype=torch.int)

print ("train feats tensor: ", train_feats_tensor.shape, " type: ", type(train_feats_tensor))
print ("train labels tensor: ", train_labels_tensor.shape, " type: ", type(train_labels_tensor))

print ("test feats tensor: ", test_feats_tensor.shape, " type: ", type(test_feats_tensor))
print ("test labels tensor: ", test_labels_tensor.shape, " type: ", type(test_labels_tensor))

print ("val feats tensor: ", val_feats_tensor.shape, " type: ", type(val_feats_tensor))
print ("val labels tensor: ", val_labels_tensor.shape, " type: ", type(val_labels_tensor))

train feats tensor:  torch.Size([3585, 1000, 128])  type:  <class 'torch.Tensor'>
train labels tensor:  torch.Size([3585, 1000, 9])  type:  <class 'torch.Tensor'>
test feats tensor:  torch.Size([497, 1000, 128])  type:  <class 'torch.Tensor'>
test labels tensor:  torch.Size([497, 1000, 9])  type:  <class 'torch.Tensor'>
val feats tensor:  torch.Size([518, 1000, 128])  type:  <class 'torch.Tensor'>
val labels tensor:  torch.Size([518, 1000, 9])  type:  <class 'torch.Tensor'>


In [25]:
# 1.1 place tensors on GPU
# if torch.cuda.is_available():
    
#     train_feats_tensor = train_feats_tensor.cuda()
#     train_labels_tensor = train_labels_tensor.cuda()
#     test_feats_tensor = test_feats_tensor.cuda()
#     test_labels_tensor = test_labels_tensor.cuda()
#     val_feats_tensor = val_feats_tensor.cuda()
#     val_labels_tensor = val_labels_tensor.cuda()

#     print ("train_feats_tensor.device: ", train_feats_tensor.get_device())
#     print ("train_labels_tensor.device: ", train_labels_tensor.get_device())
#     print ("test_feats_tensor.device: ", test_feats_tensor.get_device())
#     print ("test_labels_tensor.device: ", test_labels_tensor.get_device())
#     print ("val_feats_tensor.device: ", val_feats_tensor.get_device())
#     print ("val_labels_tensor.device: ", val_labels_tensor.get_device())

In [26]:
# 1.2 set up the data loaders
batch_size = 1

train_dataset = torch.utils.data.TensorDataset(train_feats_tensor, train_labels_tensor)
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size = batch_size, shuffle = True)

test_dataset = torch.utils.data.TensorDataset(test_feats_tensor, test_labels_tensor)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size = batch_size, shuffle = False)

test_dataset = torch.utils.data.TensorDataset(val_feats_tensor, val_labels_tensor)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size = batch_size, shuffle = False)

In [30]:
# 2.1 CNN Model Architecture

class MyCNN(nn.Module):
    def __init__(self, model_type, output_dim):
        super(MyCNN, self).__init__()
        # store model type
        self.model_type = model_type
        # batch 1:
        self.conv1 = nn.Conv2d(128, 128, kernel_size=4)
        self.relu1 = nn.ReLU()
        self.pool1 = nn.MaxPool2d(kernel_size=5)
        # batch 2
        self.conv2 = nn.Conv2d(128, 128, kernel_size=4)
        self.relu2 = nn.ReLU()
        self.pool2 = nn.MaxPool2d(kernel_size=5)
        # batch 3
        self.flatten = nn.Flatten()
        self.linear3 = nn.Linear(19968, 1024)
        self.relu3 = nn.ReLU()
        # batch 4
        self.linear4 = nn.Linear(1024, 1024)
        self.relu4 = nn.ReLU()
        # batch 5
        self.linear5 = nn.Linear(1024, output_dim)
        
    def forward(self, x):
        # reshape input
        print ("[init] x.shape: ", x.shape)
        # layer 1
        out = self.conv1(x)
        print ("[conv1] out.shape: ", out.shape)
        out = self.relu1(out)
        print ("[relu1] out.shape: ", out.shape)
        out = self.pool1(out)
        print ("[pool1] out.shape: ", out.shape)
        # layer 2
        out = self.conv2(out)
        print ("[conv2] out.shape: ", out.shape)
        out = self.relu2(out)
        print ("[relu2] out.shape: ", out.shape)
        out = self.pool2(out)
        print ("[pool2] out.shape: ", out.shape)
        # layer 3
        out = self.flatten(out)
        print ("[flat] out.shape: ", out.shape)
        out = self.linear3(out)
        print ("[line3] out.shape: ", out.shape)
        out = self.relu3(out)
        print ("[relu3] out.shape: ", out.shape)
        # layer 4
        out = self.linear4(out)
        print ("[line4] out.shape: ", out.shape)
        out = self.relu4(out)
        print ("[relu4] out.shape: ", out.shape)
        #layer 5
        out = self.linear5(out)
        print ("[line5] out.shape: ", out.shape)
        return out

In [ ]:
# 2.3 bi-LSTM Model Architecture

class bi_LSTM(nn.Module):
    def __init__(self, input_dim, hidden_dim, batch_size, output_dim, num_layers=2, model_type='LSTM'):
        super(bi_LSTM, self).__init__()
        self.model_type = model_type
        self.input_dim = input_dim
        self.hidden_dim = hidden_dim
        self.batch_size = batch_size
        self.num_layers = num_layers
        
        # initial linear hidden layer
        self.init_linear = nn.Linear(self.input_dim, self.input_dim)
        
        # LSTM layer
        self.lstm = eval('nn.' + self.model_type)(self.input_dim, self.hidden_dim, self.num_layers, batch_first=True, bidirectional=True)
        
        # output layer
        self.output_linear = nn.Linear(self.hidden_dim * self.num_layers, output_dim)
        
    def init_hidden(self, input):
        # init hidden layer
        return (torch.zeros(self.num_layers, self.batch_size, self.hidden_dim),
                torch.zeros(self.num_layers, self.batch_size, self.hidden_dim))
        
    def forward(self, input):
        #Forward pass through initial hidden layer
        linear_input = self.init_linear(input)

        # Forward pass through LSTM layer
        # shape of lstm_out: [batch_size, input_size, hidden_dim]
        # shape of self.hidden: (a, b), where a and b both
        # have shape (batch_size, num_layers, hidden_dim).
        lstm_out, self.hidden = self.lstm(linear_input)

        # Can pass on the entirety of lstm_out to the next layer if it is a seq2seq prediction
        y_pred = self.linear(lstm_out)
        return y_pred
        

In [28]:
# 3.1 helper functions for training

def test_network(model, test_loader):
    correct = 0
    total = 0
    with torch.no_grad():
        for data in test_loader:
            inputs, labels = data
            outputs = model(inputs)
            _, predicted = torch.max(outputs.data, 1)

            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    return 100 * correct / total

def print_stats(myModel, iteration_list, accuracy_list, loss_list):
    # final accuracy plot        
    plt.plot(iteration_list, accuracy_list)
    plt.title("accuracy over time")
    plt.xlabel("iterations")
    plt.ylabel("accuracy")
    plt.show()
    
    # final loss plot        
    plt.plot(iteration_list, loss_list)
    plt.title("loss over time")
    plt.xlabel("iterations")
    plt.ylabel("loss")
    plt.show()

In [31]:
# 3.0 Training

output_size = 2

# clear GPU memory
# free_gpu_cache()

kickModel = MyCNN("CNN", output_size)
kickCrit = nn.CrossEntropyLoss()
kickOpt = optim.SGD(kickModel.parameters(), lr = 0.001, momentum = 0.9)

snareModel = MyCNN("CNN", output_size)
snareCrit = nn.CrossEntropyLoss()
snareOpt = optim.SGD(snareModel.parameters(), lr = 0.001, momentum = 0.9)

hhClosedModel = MyCNN("CNN", output_size)
hhClosedCrit = nn.CrossEntropyLoss()
hhClosedOpt = optim.SGD(hhClosedModel.parameters(), lr = 0.001, momentum = 0.9)

# print ("device name: ", torch.cuda.get_device_name(0))
# print ("model.type: ", myModel.model_type)
# print ("model.device: ", next(myModel.parameters()).device)

# lists for data collection
iteration_list = []
accuracy_list = []
loss_list = []

# perform epochs
startTime = time.time()
epochs = 5
current_iteration = 0
for epoch in range(epochs):
    print ("epoch: ", epoch)
    for i, (feats, labels) in enumerate(train_loader, 0):
        feats = feats.squeeze()
        labels = labels.squeeze()
        print ("feats: ", feats.shape)
        print ("labels: ", labels.shape)
        for i in range(1000):
            frame = feats[i]
            label = labels[i]
            print ("frame: ", frame.shape)
            print ("label: ", label.shape)
            # forward pass
            kickOutputs = kickModel(frame)
            snareOutputs = snareModel(frame)
            hhClosedOutputs = hhClosedModel(frame)
            # loss
            kickLoss = kickCrit(kickOutputs, labels)
            snareLoss = snareCrit(snareOutputs, labels)
            hhClosedLoss = hhClosedCrit(hhClosedOutputs, labels)
            # backward pass and optimize
            kickOpt.zero_grad()
            kickLoss.backward()
            kickOpt.step()
            
            snareOpt.zero_grad()
            snareLoss.backward()
            snareOpt.step()
            
            hhClosedOpt.zero_grad()
            hhClosedLoss.backward()
            hhClosedOpt.step()
            # calculate accuracy
            if current_iteration % 100 == 0:  
                test_accuracy = test_network(myModel, test_loader)
                accuracy_list.append(test_accuracy)
                iteration_list.append(current_iteration)
                loss_list.append(loss.item())
            # print stats during training
            if current_iteration % 100 == 0:
                test_accuracy = test_network(myModel, test_loader)
                print(f'\t iteration: {current_iteration}\t loss: {loss.item():.3f}\t accuracy: {test_accuracy:.3f} %')
                
            current_iteration += 1         
# print out stats
print_stats(myModel, iteration_list, accuracy_list, loss_list)
print ("time elapsed: ", round((time.time() - startTime), 2), " sec")

epoch:  0
feats:  torch.Size([1000, 128])
labels:  torch.Size([1000, 9])
frame:  torch.Size([128])
label:  torch.Size([9])
[init] x.shape:  torch.Size([128])


RuntimeError: Expected 3D (unbatched) or 4D (batched) input to conv2d, but got input of size: [128]